Ref: https://towardsdatascience.com/weight-initialization-in-neural-networks-a-journey-from-the-basics-to-kaiming-954fb9b47c79

James Dellinger's paper is really explanable to understand the initialization topic. I thank to him for this great paper. I copy and paste all text and codes from this paper. You can check the website for more information. 


In [1]:
from torch.nn import init
import torch
import math
import numpy as np

In [2]:
x = torch.randn(512)
print('x mean: ', x.mean(), 'x std: ', x.std())

x mean:  tensor(-0.0782) x std:  tensor(0.9463)


Let’s also pretend that we have a simple 100-layer network with no activations ,
and that each layer has a matrix a that contains the layer’s weights.

It turns out that initializing the values of layer weights from the same standard NORMAL distribution to which we scaled our inputs is NEVER a good idea.

In [3]:
for i in range(100):
    a = torch.randn(512, 512)
    x = torch.matmul(a, x)

print('x mean: ', x.mean(), 'x std: ', x.std())

x mean:  tensor(nan) x std:  tensor(nan)


Why are they nan?

In [4]:
x = torch.randn(512)
for i in range(100):
    a = torch.randn(512, 512)
    x = torch.matmul(a, x)
    if torch.isnan(x.std()): break

print(i)

28


The activation outputs exploded within 27 of our network’s layers.

We clearly initialized our weights to be too large.

Let's use small weights for initialization by multiply 0.01

In [5]:
x = torch.randn(512)
for i in range(100):
    a = torch.randn(512, 512) * 0.01
    x = torch.matmul(a, x)

print('x mean: ', x.mean(), 'x std: ', x.std())

x mean:  tensor(0.) x std:  tensor(0.)


They are vanished :(

The matrix product of our inputs x and weight matrix a that we initialized from a standard normal distribution will, 
on average, have a standard deviation very close to the square root of the number of input connections, 
which in our example is sqrt(512).

In [7]:
mean, var = 0.0, 0.0
y_sum, y_var = 0.0, 0.0
for i in range(10000):
    x = torch.randn(512)
    a = torch.randn(512, 512)
    y = torch.matmul(a, x)
    mean += y.mean().item()
    var += y.pow(2).mean().item()
    y_sum += y.sum().item()

print('mean: ', mean/10000, 'std: ', math.sqrt(var/10000))

print('sqrt of number of inputs: ', math.sqrt(512))

mean:  0.0016365561494603752 std:  22.617465906573933
sqrt of number of inputs:  22.627416997969522


In [8]:
y_sum/10000/512 # for 512 inputs we do the matrix mult. for 10000 times. 

0.0016365561494603752

In [17]:
# The same for 1 input:
mean_1, var_1, y_sum_1 = 0.0, 0.0, 0.0
for i in range(10000):
    x = torch.randn(1)
    a = torch.randn(1)
    y = torch.matmul(a, x)
    mean_1 += y.mean().item()
    var_1 += y.pow(2).mean().item()
    y_sum_1 += y.sum().item()


print('mean for 1 input: ', mean_1/10000, 'std for 1 input: ', math.sqrt(var_1/10000))
# mean for 1 input:  -0.00942440634764862 std for 1 input:  1.0046721221399195

mean for 1 input:  0.010211637498661753 std for 1 input:  0.9880182666932059


It then follows that the sum of these 512 products would have a mean of 0,  variance of 512, and therefore a standard deviation of √512.

What we’d like is for each layer’s outputs to have a standard deviation of about 1. 

If we first scale the weight matrix a by dividing all its randomly chosen values by √512, the element-wise multiplication that fills in one element of the outputs y would now, on average, have a variance of only 1/√512


In [15]:
mean, var = 0.0, 0.0
for i in range(10000):
    x = torch.randn(1)
    a = torch.randn(1) * math.sqrt(1./512)
    y = torch.matmul(a, x)
    mean += y.item()
    var += y.pow(2).item()

print('mean: ', mean/10000, 'var: ', var/10000)

print('check the var ', 1/512)

mean:  -0.0003472797142562627 var:  0.0018953190911959602
check the var  0.001953125


Using 1/input --> achieve 1 std:

In [16]:
mean, var = 0.0, 0.0
y_sum, y_var = 0.0, 0.0
for i in range(10000):
    x = torch.randn(512)
    a = torch.randn(512, 512) * math.sqrt(1/512)
    y = torch.matmul(a, x)
    mean += y.mean().item()
    var += y.pow(2).mean().item()
    y_sum += y.sum().item()

print('mean: ', mean/10000, 'std: ', math.sqrt(var/10000))

mean:  -0.00012715190263697877 std:  0.999678050607976


In [19]:
x = torch.randn(512)

for i in range(100):  # 100 layer network
    a = torch.randn(512, 512) * math.sqrt(1/512)
    x = torch.matmul(a, x)

print('mean: ', x.mean(), 'std: ', x.std())

mean:  tensor(-0.1187) std:  tensor(1.1257)


Success! Our layer outputs neither exploded nor vanished, even after 100 of our hypothetical layers.


## Xavier Initialization

Let's use activation functions

In [20]:
def tanh(x):
  return torch.tanh(x)

In [22]:
x = torch.randn(512)

for i in range(100):  # 100 layer network
    a = torch.randn(512, 512) * math.sqrt(1/512)
    x = tanh(torch.matmul(a, x))

print('mean: ', x.mean(), 'std: ', x.std())

mean:  tensor(-0.0036) std:  tensor(0.0933)


The standard deviation of activation outputs of the 100th layer is down to about 0.06. This is definitely on the small side, but at least activations haven’t totally vanished!

When Xavier Glorot and Yoshua Bengio published their landmark paper titled Understanding the difficulty of training deep feedforward neural networks, the “commonly used heuristic” to which they compared their experiments was that of initializing weights from a uniform distribution in [-1,1] and then scaling by 1/√n.

In [23]:
x = torch.randn(512)

for i in range(100):  # 100 layer network
    a = torch.Tensor(512, 512).uniform_(-1,1) * math.sqrt(1/512)
    x = tanh(torch.matmul(a, x))

print('mean: ', x.mean(), 'std: ', x.std())

mean:  tensor(-5.3748e-26) std:  tensor(8.7262e-25)


 They’re just about as good as vanished with uniform distribution :(

Xavier initialization sets a layer’s weights to values chosen from a random uniform distribution that’s bounded between : +- √6 / (√ni + ni+1)

- nᵢ: the number of incoming network connections, or “fan-in,” to the layer. 

- nᵢ₊₁: the number of outgoing network connections from that layer, also known as the “fan-out.”

Let's use Xavier Initialization for our 100 layer network with input 512:

In [24]:
def xavier(m,h):
  return torch.Tensor(m,h).uniform_(-1,1) * math.sqrt(6./(m+h))

In [25]:
x = torch.randn(512)
m = 512
h = 512

for i in range(100):  # 100 layer network
    a = xavier(m,h)
    x = tanh(torch.matmul(a, x))

print('mean: ', x.mean(), 'std: ', x.std())

mean:  tensor(0.0047) std:  tensor(0.0729)


In our experimental network, Xavier initialization performs pretty identical to the home-grown method that we derived earlier, where we sampled values from a random normal distribution and scaled by the square root of number of incoming network connections, n.

## Kaiming Initialization


When using activation functions that are symmetric about zero and have outputs inside [-1,1], such as softsign and tanh:

we’d want the activation outputs of each layer to have a MEAN of 0 and a STD around 1, on average. 


This is precisely what our home-grown method and Xavier both enable.

But what if we’re using ReLU activation functions? Would it still make sense to want to scale random initial weight values in the same way?

Let's use ReLu:


In [26]:
def relu(x):
  return x.clamp_min(0.)

In [27]:
mean, var= 0., 0.
y_sum, y_var = 0.0, 0.0

for i in range(10000):

    x = torch.randn(512)
    a = torch.randn(512, 512)
    y = relu(torch.matmul(a, x))

    mean += y.mean().item()
    var += y.pow(2).mean().item()
    y_sum += y.sum().item()

print('mean: ', mean/10000, 'std: ', math.sqrt(var/10000))


mean:  9.00831508398056 std:  15.981985173101688


In [28]:
math.sqrt(512/2)

16.0

when using a ReLU activation, a single layer will, on average have standard deviation that’s very close to the square root of the number of input connections, divided by the square root of two, or √512/√2 in our example.

In [29]:
mean, var= 0., 0.
y_sum, y_var = 0.0, 0.0

for i in range(10000):

    x = torch.randn(512)
    a = torch.randn(512, 512) * math.sqrt(2/512)
    y = relu(torch.matmul(a, x))

    mean += y.mean().item()
    var += y.pow(2).mean().item()
    y_sum += y.sum().item()

print('mean: ', mean/10000, 'std: ', math.sqrt(var/10000))


mean:  0.5641567468941212 std:  1.0005376670522719


Std is now close to 1. 

Why this is so important?

Keeping the standard deviation of layers’ activations around 1 will allow us to stack several more layers in a deep neural network without gradients exploding or vanishing.

In their 2015 paper, He et. al. demonstrated that deep networks (e.g. a 22-layer CNN) would converge much earlier if the following input weight initialization strategy is employed:

- Create a tensor with the dimensions appropriate for a weight matrix at a given layer, and populate it with numbers randomly chosen from a standard normal distribution.

- Multiply each randomly chosen number by √2/√n where n is the number of incoming connections coming into a given layer from the previous layer’s output (also known as the “fan-in”).

- Bias tensors are initialized to zero.

In [31]:
def kaiming(m,h):
  return torch.randn(m,h) * math.sqrt(2. / m)

In [32]:
x = torch.randn(512)
m = 512
h = 512

for i in range(100):  # 100 layer network
    a = kaiming(m,h)
    x = relu(torch.matmul(a, x))

print('mean: ', x.mean(), 'std: ', x.std())

mean:  tensor(0.1514) std:  tensor(0.2260)


What if we use xavier instead?

In [33]:
x = torch.randn(512)
m = 512
h = 512

for i in range(100):  # 100 layer network
    a = xavier(m,h)
    x = relu(torch.matmul(a, x))

print('mean: ', x.mean(), 'std: ', x.std())

mean:  tensor(4.8009e-16) std:  tensor(6.4892e-16)


Vanished!